<a href="https://colab.research.google.com/github/FuturisticLover123/learning/blob/main/%D0%A2%D1%80%D0%B5%D0%BD%D0%B8%D1%80%D0%BE%D0%B2%D0%BA%D0%B8%20%D0%BF%D0%BE%20ML%203.0/task5/pupurum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [207]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [297]:
#train = pd.read_csv("/content/train.csv")
train = pd.read_excel("/content/Pistachio_16_Features_Dataset.xlsx")
test = pd.read_csv("/content/true_test.csv")

In [298]:
train["Class"] = train["Class"].map({"Kirmizi_Pistachio": 1, "Siit_Pistachio": 0})

In [294]:
data = pd.read_csv("/content/train.csv")
train = data.sample(frac=0.8, random_state=42)
test = data.drop(train.index)


In [299]:
X_test, y_test = test.drop("Class", axis=1), test["Class"]
X_train, y_train = train.drop("Class", axis=1), train["Class"]

In [300]:
from sklearn.ensemble import IsolationForest
iso = IsolationForest(random_state=0, contamination=0.05)
clean = iso.fit_predict(X_train,y_train)

X_train = X_train[clean != -1]
y_train = y_train[clean != -1]

In [301]:
#балансируем выборку
from imblearn.over_sampling import RandomOverSampler
sampler = RandomOverSampler(random_state=42)
X_train, y_train = sampler.fit_resample(X_train, y_train)

In [302]:
from sklearn.feature_selection import SelectKBest, chi2
skf = SelectKBest(k=10, score_func=chi2)
skf.fit(X_train,y_train)
x = X_train.iloc[:, skf.get_support()]

In [303]:

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [277]:
!pip install optuna
!pip install catboost

In [278]:
import optuna
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


def objective(trial):
    model = CatBoostClassifier(
        iterations=1000,
        learning_rate=trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        depth=trial.suggest_int('depth', 4, 10),

        l2_leaf_reg=trial.suggest_float('l2_leaf_reg', 1e-5, 10, log=True),
        rsm=trial.suggest_float('rsm', 0.5, 1.0),
        bagging_temperature=trial.suggest_float('bagging_temperature', 0.0, 10.0),

        early_stopping_rounds=2,


        grow_policy=trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise']),
        min_data_in_leaf=trial.suggest_int('min_data_in_leaf', 1, 20),

        verbose=0,
        random_state=42
    )

    model.fit(
        X_train, y_train,
        eval_set=(X_val, y_val),
        use_best_model=True
    )

    y_pred = model.predict(X_val)
    return accuracy_score(y_val, y_pred)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)


best_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=study.best_params['learning_rate'],
    depth=study.best_params['depth'],
    verbose=0
)
best_model.fit(X_train, y_train)


accuracy = accuracy_score(y_test, best_model.predict(X_test))
print(f"Лучшая точность: {accuracy:.4f}")
print(f"Лучшие параметры: {study.best_params}")

[I 2025-04-24 19:40:07,316] A new study created in memory with name: no-name-fbf8b47d-02a9-4126-9530-28ada5e6d67d
[I 2025-04-24 19:40:07,820] Trial 0 finished with value: 0.8983050847457628 and parameters: {'learning_rate': 0.08449115870432951, 'depth': 7, 'l2_leaf_reg': 0.014600991217095915, 'rsm': 0.8399934261892148, 'bagging_temperature': 9.38960558463519, 'grow_policy': 'SymmetricTree', 'min_data_in_leaf': 4}. Best is trial 0 with value: 0.8983050847457628.
[I 2025-04-24 19:40:07,979] Trial 1 finished with value: 0.8898305084745762 and parameters: {'learning_rate': 0.1910302547635859, 'depth': 6, 'l2_leaf_reg': 0.009559086319838879, 'rsm': 0.7903816129753097, 'bagging_temperature': 6.178662605904725, 'grow_policy': 'SymmetricTree', 'min_data_in_leaf': 6}. Best is trial 0 with value: 0.8983050847457628.
[I 2025-04-24 19:40:08,176] Trial 2 finished with value: 0.9004237288135594 and parameters: {'learning_rate': 0.11533099816287033, 'depth': 7, 'l2_leaf_reg': 0.003107694690492959, 'r

Лучшая точность: 0.9605
Лучшие параметры: {'learning_rate': 0.08299777428137957, 'depth': 5, 'l2_leaf_reg': 0.11452452345285627, 'rsm': 0.734767277846577, 'bagging_temperature': 7.74126202535618, 'grow_policy': 'Depthwise', 'min_data_in_leaf': 8}


In [279]:
import lightgbm as lgb
from sklearn.metrics import accuracy_score

def objective_lightgbm(trial):
    params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'boosting_type': 'gbdt',
        'n_estimators': 1000,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 20, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 20),

        'reg_alpha': trial.suggest_float('reg_alpha', 1e-5, 10, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-5, 10, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),

        'verbosity': -1,
        'random_state': 42
    }

    model = lgb.LGBMClassifier(**params)
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=False)]
    )

    y_pred = model.predict(X_val)
    return accuracy_score(y_val, y_pred)

study_lightgbm = optuna.create_study(direction='maximize')
study_lightgbm.optimize(objective_lightgbm, n_trials=50)


[I 2025-04-24 19:42:01,087] A new study created in memory with name: no-name-ce917807-a40f-4572-adee-749af5ff2da5
[I 2025-04-24 19:42:01,691] Trial 0 finished with value: 0.9046610169491526 and parameters: {'learning_rate': 0.024143302422279335, 'num_leaves': 219, 'max_depth': 9, 'min_child_samples': 12, 'reg_alpha': 0.05554412346213887, 'reg_lambda': 0.002097011696061595, 'feature_fraction': 0.5565760374691778, 'bagging_fraction': 0.8843902590290533, 'bagging_freq': 5}. Best is trial 0 with value: 0.9046610169491526.
[I 2025-04-24 19:42:02,174] Trial 1 finished with value: 0.9004237288135594 and parameters: {'learning_rate': 0.04486909011755782, 'num_leaves': 208, 'max_depth': 12, 'min_child_samples': 5, 'reg_alpha': 0.008793323259823528, 'reg_lambda': 0.0003822167753745501, 'feature_fraction': 0.5184581208119978, 'bagging_fraction': 0.9980180494370314, 'bagging_freq': 8}. Best is trial 0 with value: 0.9046610169491526.
[I 2025-04-24 19:42:02,331] Trial 2 finished with value: 0.894067

In [280]:
import xgboost as xgb
from sklearn.metrics import accuracy_score

def objective_xgboost(trial):
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'booster': 'gbtree',
        'n_estimators': 1000,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-5, 10, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-5, 10, log=True),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'early_stopping_rounds': 50,
        'verbosity': 0,
        'random_state': 42
    }

    model = xgb.XGBClassifier(**params)
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        verbose=False
    )

    y_pred = model.predict(X_val)
    return accuracy_score(y_val, y_pred)

study_xgboost = optuna.create_study(direction='maximize')
study_xgboost.optimize(objective_xgboost, n_trials=50)

[I 2025-04-24 19:42:27,515] A new study created in memory with name: no-name-e1f67294-6232-45e9-b797-e863857384d4
[I 2025-04-24 19:42:33,452] Trial 0 finished with value: 0.8961864406779662 and parameters: {'learning_rate': 0.010006313169607258, 'max_depth': 3, 'min_child_weight': 4, 'reg_alpha': 0.004939411060728005, 'reg_lambda': 3.558692943279335e-05, 'subsample': 0.9556649410918299, 'colsample_bytree': 0.7979022761948658, 'gamma': 4.872537037326397}. Best is trial 0 with value: 0.8961864406779662.
[I 2025-04-24 19:42:33,661] Trial 1 finished with value: 0.9004237288135594 and parameters: {'learning_rate': 0.11809632496736236, 'max_depth': 6, 'min_child_weight': 1, 'reg_alpha': 0.0009642982576453782, 'reg_lambda': 0.29601453713306375, 'subsample': 0.6887066828561372, 'colsample_bytree': 0.6620515657140045, 'gamma': 1.440479347143716}. Best is trial 1 with value: 0.9004237288135594.
[I 2025-04-24 19:42:33,890] Trial 2 finished with value: 0.9004237288135594 and parameters: {'learning

In [281]:
best_lgb_params = study_lightgbm.best_params
best_lgb_params.update({
    'objective': 'binary',
    'metric': 'binary_logloss',
    'n_estimators': 1000,
    'verbosity': -1,
    'random_state': 42
})

lgb_model = lgb.LGBMClassifier(**best_lgb_params)

best_xgb_params = study_xgboost.best_params
best_xgb_params.update({
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'n_estimators': 1000,
    'early_stopping_rounds': 50,
    'verbosity': 0,
    'random_state': 42
})

xgb_model = xgb.XGBClassifier(**best_xgb_params)

from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, classification_report


best_cat_params = study.best_params
best_cat_params.update({
    'iterations': 1000,
    'verbose': False,
    'random_state': 42,
    'early_stopping_rounds': 50
})

cat_model = CatBoostClassifier(**best_cat_params)


In [282]:

lgb_model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    callbacks=[lgb.early_stopping(stopping_rounds=50, verbose=False)]
)

lgb_pred = lgb_model.predict(X_val)
print("LightGBM Accuracy:", accuracy_score(y_val, lgb_pred))
print("LightGBM Best params:", best_lgb_params)

LightGBM Accuracy: 0.9152542372881356
LightGBM Best params: {'learning_rate': 0.1657437807117286, 'num_leaves': 125, 'max_depth': 10, 'min_child_samples': 18, 'reg_alpha': 0.0624088862915544, 'reg_lambda': 0.0010395845773651304, 'feature_fraction': 0.5810815599857436, 'bagging_fraction': 0.8957906820699924, 'bagging_freq': 7, 'objective': 'binary', 'metric': 'binary_logloss', 'n_estimators': 1000, 'verbosity': -1, 'random_state': 42}


In [283]:

xgb_model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=False
)


xgb_pred = xgb_model.predict(X_val)
print("\nXGBoost Accuracy:", accuracy_score(y_val, xgb_pred))
print("XGBoost Best params:", best_xgb_params)


XGBoost Accuracy: 0.9067796610169492
XGBoost Best params: {'learning_rate': 0.017781017212363446, 'max_depth': 4, 'min_child_weight': 7, 'reg_alpha': 2.0149293705021204e-05, 'reg_lambda': 0.2305773029216583, 'subsample': 0.9158951009841575, 'colsample_bytree': 0.8587333186185291, 'gamma': 4.917998535342417, 'objective': 'binary:logistic', 'eval_metric': 'logloss', 'n_estimators': 1000, 'early_stopping_rounds': 50, 'verbosity': 0, 'random_state': 42}


In [284]:

cat_model.fit(
    X_train, y_train,
    eval_set=(X_val, y_val),
    use_best_model=True,
    plot=False
)

cat_pred = cat_model.predict(X_val)
print("\nCatBoost Accuracy:", accuracy_score(y_val, cat_pred))
print("CatBoost Best params:", best_cat_params)



CatBoost Accuracy: 0.9004237288135594
CatBoost Best params: {'learning_rate': 0.08299777428137957, 'depth': 5, 'l2_leaf_reg': 0.11452452345285627, 'rsm': 0.734767277846577, 'bagging_temperature': 7.74126202535618, 'grow_policy': 'Depthwise', 'min_data_in_leaf': 8, 'iterations': 1000, 'verbose': False, 'random_state': 42, 'early_stopping_rounds': 50}


In [252]:
from sklearn.ensemble import VotingClassifier
lgb_for_ensemble = lgb.LGBMClassifier(**{**best_lgb_params, 'early_stopping_rounds': None})
xgb_for_ensemble = xgb.XGBClassifier(**{**best_xgb_params, 'early_stopping_rounds': None})
cat_for_ensemble = CatBoostClassifier(**{**best_cat_params, 'early_stopping_rounds': None})

lgb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)])
xgb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=False)
cat_model.fit(X_train, y_train, eval_set=(X_val, y_val), verbose=False)

ensemble = VotingClassifier(
    estimators=[
        ('lgb', lgb_for_ensemble),
        ('xgb', xgb_for_ensemble),
        ('cat', cat_for_ensemble)
    ],
    voting='soft'
)

ensemble.fit(X_train, y_train)

VotingClassifier(estimators=[('lgb',
                              LGBMClassifier(bagging_fraction=0.9988142589202635,
                                             bagging_freq=1,
                                             early_stopping_rounds=None,
                                             feature_fraction=0.5656819252118983,
                                             learning_rate=0.1630434483082602,
                                             max_depth=9,
                                             metric='binary_logloss',
                                             min_child_samples=12,
                                             n_estimators=1000, num_leaves=50,
                                             objective='binary',
                                             random_state=42,
                                             reg_alpha=0.0089211209163905...
                                            learning_rate=0.26327541032792284,
                                            max_bin=None,
                                            max_cat_threshold=None,
                                            max_cat_to_onehot=None,
                                            max_delta_step=None, max_depth=5,
                                            max_leaves=None, min_child_weight=5,
                                            missing=nan,
                                            monotone_constraints=None,
                                            multi_strategy=None,
                                            n_estimators=1000, n_jobs=None,
                                            num_parallel_tree=None,
                                            random_state=42, ...)),
                             ('cat',
                              <catboost.core.CatBoostClassifier object at 0x7a8b6851b9d0>)],
                 voting='soft')

In [285]:
ensemble_pred = ensemble.predict(X_val)
print("\nEnsemble Accuracy:", accuracy_score(y_val, ensemble_pred))


Ensemble Accuracy: 0.9173728813559322


In [286]:
ensemble_pred = ensemble.predict(X_test)
print("\nEnsemble Accuracy:", accuracy_score(y_test, ensemble_pred))


Ensemble Accuracy: 0.9720930232558139


In [306]:
test = pd.read_csv("/content/test.csv")

In [309]:
test.rename(columns={name:name.upper() for name in test.columns}, inplace=True)

In [310]:
ensemble_pred = ensemble.predict(test)

In [312]:
pd.DataFrame(ensemble_pred).to_csv("last_pred.csv",index=False, header = False)